In [1]:
import pandas as pd

Process CORUM data that was downloaded from `https://github.com/broadinstitute/2022_PERISCOPE/tree/main/common_files/CORUM_humanComplexes.txt`. It is not clear where this was downloaded from.

### Read CORUM data

In [2]:
corum_df = (
    pd.read_csv('input/CORUM_humanComplexes.txt', sep='\t')
)
corum_df.head()

,ComplexID,ComplexName,Organism,Synonyms,Cell line,subunits(UniProt IDs),subunits(Entrez IDs),Protein complex purification method,GO ID,GO description,FunCat ID,FunCat description,PubMed ID,Complex comment,Disease comment,Subunits comment,SWISSPROT organism,subunits(Protein name),subunits(Gene name),subunits(Gene name syn)
0,1,BCL6-HDAC4 complex,Human,None,U2OS osteosarcoma-derived UTA-L cells,P41182;P56524,604;9759,MI:0007-anti tag coimmunoprecipitation,GO:0006265;GO:0045892;GO:0051276;GO:0030183;GO...,DNA topological change;negative regulation of ...,10.01.09.05;11.02.03.04.03;42.10.03;43.03.07.0...,DNA conformation modification (e.g. chromatin)...,11929873,Transcriptional repression by BCL6 is thought ...,None,None,Homo sapiens (Human);Homo sapiens (Human),B-cell lymphoma 6 protein;Histone deacetylase 4,BCL6;HDAC4,BCL5 LAZ3 ZBTB27 ZNF51;KIAA0288
1,2,BCL6-HDAC5 complex,Human,None,U2OS osteosarcoma-derived UTA-L cells,P41182;Q9UQL6,604;10014,MI:0007-anti tag coimmunoprecipitation,GO:0006265;GO:0045892;GO:0051276;GO:0030183;GO...,DNA topological change;negative regulation of ...,10.01.09.05;11.02.03.04.03;42.10.03;43.03.07.0...,DNA conformation modification (e.g. chromatin)...,11929873,Transcriptional repression by BCL6 is thought ...,None,None,Homo sapiens (Human);Homo sapiens (Human),B-cell lymphoma 6 protein;Histone deacetylase 5,BCL6;HDAC5,BCL5 LAZ3 ZBTB27 ZNF51;KIAA0600
2,3,BCL6-HDAC7 complex,Human,None,U2OS osteosarcoma-derived UTA-L cells,P41182;Q8WUI4,604;51564,MI:0007-anti tag coimmunoprecipitation,GO:0006265;GO:0045892;GO:0051276;GO:0030183;GO...,DNA topological change;negative regulation of ...,10.01.09.05;11.02.03.04.03;42.10.03;43.03.07.0...,DNA conformation modification (e.g. chromatin)...,11929873,Transcriptional repression by BCL6 is thought ...,None,None,Homo sapiens (Human);Homo sapiens (Human),B-cell lymphoma 6 protein;Histone deacetylase 7,BCL6;HDAC7,BCL5 LAZ3 ZBTB27 ZNF51;HDAC7A
3,4,Multisubunit ACTR coactivator complex,Human,None,different human tissues and cancer cell lines,Q09472;Q92793;Q92831;Q9Y6Q9,2033;1387;8850;8202,MI:0004-affinity chromatography technology;MI:...,GO:0030374;GO:0004402,nuclear receptor coactivator activity;histone ...,None,None,9267036,Cofactor ACTR binds directly nuclear receptors...,None,None,Homo sapiens (Human);Homo sapiens (Human);Homo...,Histone acetyltransferase p300;CREB-binding pr...,EP300;CREBBP;KAT2B;NCOA3,P300;CBP;PCAF;AIB1 BHLHE42 RAC3 TRAM1
4,10,Condensin I complex,Human,13S condensin complex,HeLa cells,O95347;Q15003;Q15021;Q9BPX3;Q9NTJ3,10592;23397;9918;64151;10051,MI:0004-affinity chromatography technology;MI:...,GO:0000279;GO:0030261;GO:0007059;GO:0051276;GO...,M phase;chromosome condensation;chromosome seg...,10.03.01.01.11;10.03.04.03;10.03.04.05;42.10.0...,M phase;chromosome condensation;chromosome seg...,11136719,None,None,None,Homo sapiens (Human);Homo sapiens (Human);Homo...,Structural maintenance of chromosomes protein ...,SMC2;NCAPH;NCAPD2;NCAPG;SMC4,CAPE SMC2L1;BRRN BRRN1 CAPH KIAA0074;CAPD2 CNA...


### Create `|` separated complex names for each gene

In [3]:
corum_refactor_df = (
    corum_df[["ComplexName", "subunits(Gene name)"]].copy()
    .rename(columns={"subunits(Gene name)": "genes"})
    .assign(gene = lambda x: x.genes.str.split(';'))
    .explode("gene")
    .reset_index(drop=True)
    .groupby("gene").ComplexName.apply(lambda x: '|'.join(x))
    .reset_index()
)

corum_refactor_df.head()

,gene,ComplexName
0,A2M,A2M-BCHE complex
1,AAAS,Nuclear pore complex
2,AATF,AATF-NGDN-NOL10 complex
3,ABCB1,ABCB1-ANXA2-RACK1-SRC complex|ABCB1-PPP2R3C-TF...
4,ABCB11,ABCB11-CHMP5 complex


### Write to file

In [4]:
corum_refactor_df.to_csv('output/CORUM_proteincomplex_annotations.tsv', sep='\t', index=False)